## Import tensorflow, keras modules

In [ ]:
#from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds  # For loading datasets
import numpy as np
import matplotlib.pyplot as plt     # For plotting images
import cv2                          # For resizing images
from keras import Model

print(tf.version.VERSION)
print(tf.keras.__version__)

In [ ]:
#testing the GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Import  MNIST Dataset

In [ ]:
mnist = tf.keras.datasets.mnist
# More dataset choices here: https://www.tensorflow.org/api_docs/python/tf/keras/datasets
# input image dimensions
img_x, img_y, img_z = 28, 28, 1

In [ ]:
# Load training data, labels; and testing data and their true labels
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# reshape the data into a 4D tensor - (sample_number, x_img_size, y_img_size, num_channels)
# because the MNIST is greyscale, we only have a single channel - RGB colour images would have 3
train_images = train_images.reshape(train_images.shape[0], img_x, img_y, 1)
test_images  = test_images.reshape(test_images.shape[0], img_x, img_y, 1)
input_shape = (img_x, img_y, img_z)

# Normalize input between 0 and 1
# Very important
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# Check shapes of train_images, train_labels etc
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

### Visualize the dataset

In [ ]:
# For printing, we name each of the 10 classes below
class_names = ['0', '1', '2', '3', '4','5', '6', '7', '8', '9']

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i].reshape(img_x, img_y), cmap=plt.cm.binary)
    #print(train_labels[i][0])
    plt.xlabel(class_names[train_labels[i]])

## (a) Create and train Lenet-5 Using Keras API on MNIST dataset **[3 points]**

In [ ]:
'''
Create a NN with 1 input layer
1 conv2D layer, 6 filters, 5x5 filter size, stride = (1, 1), activation tanh, use padding='same' argument (check it on https://keras.io/api/layers/convolution_layers/convolution2d/ )
1 AveragePooling2D layer (use default arguments in https://www.tensorflow.org/api_docs/python/tf/keras/layers/AveragePooling2D )
1 conv2D layer, 16 Filters, 5x5 filter size, stride = (1, 1), activation tanh, padding='valid'
1 AveragePooling2D layer (Default arguements)
1 conv2D layer, 120 filters, 5x5 filter size, stride = (1, 1), activation tanh, padding='valid'
Flatten layer
1 Dense layer, 84 units, tanh activation
1 output layer
'''
input_shape = train_images[0].shape
model = '<<Write your code here>>



In [ ]:
# Compile the model with appropriate Loss function
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model on MNIST dataset
epochs = '<<your value>>
batch_size = '<<your value>>
model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs)

##(b) Check Accuracy on Test Data **[0.5 point]**

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
# Try to get 90% or more accuracy
print('Test accuracy:', test_acc)

### Visualize predictions on test data

In [ ]:
# Get all predictions for test data
predictions = model.predict(test_images)

In [ ]:
# Code to visualize predictions
# Incorrect predictions are highlighted in red
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i].reshape(28, 28), cmap=plt.cm.binary)
    predicted_label = np.argmax(predictions[i])
    true_label = test_labels[i]
    if predicted_label == true_label:
      color = 'green'
    else:
      color = 'red'
    plt.xlabel("{} ({})".format(class_names[predicted_label], 
                                  class_names[true_label]),
                                  color=color)

##(c) Download binary_alpha_digits dataset using tfds, split dataset **[1 point]**

In [ ]:
## write your code here
dataset_name = "binary_alpha_digits"

ds_images, ds_labels = '<<Code for downloading binary_alpha_digits>>

In [ ]:
## Split dataset into 20% testing and 80% training

test_size = 0.2   # fraction of test data

'<<Write additional code as required>>

train_images = '<<your code>>
train_labels = '<<your code>>
test_images = '<<your code>>
test_labels = '<<your code>>

# Check training, testing data size
print(train_images.shape)
print(test_images.shape)

### Visualize data

In [ ]:
# Code to visualize predictions
# Incorrect predictions are highlighted in red
import matplotlib.pyplot as plt
%matplotlib inline
class_names = ['0', '1', '2', '3', '4','5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i].reshape(20, 16), cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])

## (d) Upscale training, testing data to MNIST image size (28, 28, 1) **[2 points]**

In [ ]:
'Upscale Data'
newSize = 28

# create a numpy array for storing upscaled training images
train_upscale = np.zeros((train_images.shape[0], newSize, newSize, 1))
'<<Write code for upscaling training data>> Look at function cv2.resize in opencv https://pythonexamples.org/python-opencv-cv2-resize-image/ 
print(train_upscale.shape)

# create a numpy array for storing upscaled testing images
test_upscale = np.zeros((test_images.shape[0], newSize, newSize, 1))
'<<Write code for upscaling testing data>>
print(test_upscale.shape)

In [ ]:
%matplotlib inline
# Visualize upscaled images
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_upscale[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])

##(e) Transfer learning-- Remove Last layer from your trained LeNet **[0.5 points]**



In [ ]:
## You can decide whether to train the whole network again or fix layer weights from the MNIST-trained network
## Check link: https://keras.io/getting_started/faq/#how-can-i-freeze-keras-layers 
'<<Write code for either freezing or training all layers from scratch>>

## Code for removing last layer
'<<Write code>>


##(f) Transfer learning-- Add new layers to LeNet **[1.5 points]**


In [ ]:
## Add one or more hidden layer
## Add output layer
'<<Write code here>>

In [ ]:
# Compile the model with appropriate Loss function
model.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

##(g) Train the model and show accuracy on the testing dataset (test_upscale) **[1.5 point]**

In [ ]:
## Your code here